# 🚁 Drone Resupply Benchmark - Google Colab

Notebook này cho phép bạn chạy benchmark bài toán Drone Resupply trên Google Colab.

## Hướng dẫn sử dụng:
1. **Upload project**: Tải file ZIP của project lên Google Drive
2. **Chạy các cell theo thứ tự** từ trên xuống dưới
3. **Kết quả**: File CSV và solutions sẽ được lưu trong thư mục output

## 1. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. Upload và giải nén Project

**Cách 1**: Nếu bạn đã upload file `newsolve.zip` lên Drive, chỉ định đường dẫn:

In [ ]:
# Đường dẫn tới file ZIP trên Drive (thay đổi nếu cần)
ZIP_PATH = '/content/drive/MyDrive/newsolve.zip'

# Giải nén
!unzip -o "{ZIP_PATH}" -d /content/newsolve
print("Giải nén hoàn tất!")

**Cách 2**: Upload trực tiếp từ máy tính (nếu chưa có trên Drive):

In [ ]:
# Uncomment để upload trực tiếp
# from google.colab import files
# uploaded = files.upload()  # Chọn file newsolve.zip
# !unzip -o newsolve.zip -d /content/newsolve

## 3. Cài đặt Dependencies

In [ ]:
%cd /content/newsolve
!pip install pandas

## 4. Kiểm tra cấu trúc thư mục

In [ ]:
import os
print("Thư mục hiện tại:", os.getcwd())
print("\nNội dung:")
!ls -la
print("\nThư mục data:")
!ls data/ 2>/dev/null || echo "Không tìm thấy thư mục data"

## 5. Import các module cần thiết

In [ ]:
import sys
sys.path.insert(0, '/content/newsolve/src')

import os
import time
import glob
import statistics
import pandas as pd
import concurrent.futures
from typing import Dict

from models import load_problem
from initializer import SolutionInitializer
from optimization import TabuSearch

print("Import thành công!")

## 6. Định nghĩa hàm Benchmark

In [ ]:
def process_instance(filepath: str, solution_output_dir: str, num_rounds: int, num_iters: int) -> Dict:
    """
    Xử lý một instance nhiều lần.
    """
    try:
        instance_name = os.path.basename(filepath)
        problem = load_problem(filepath)
        
        makespans = []
        runtimes = []
        best_sol_object = None
        best_makespan_instance = float('inf')
        
        for r in range(num_rounds):
            start_t = time.time()
            
            # Init
            initializer = SolutionInitializer(problem)
            init_sol = initializer.initialize4bench()
            
            # Optimization
            tabu = TabuSearch(problem, init_sol)
            final_sol = tabu.solve4bench(max_iterations=num_iters, tabu_tenure=20)
            
            end_t = time.time()
            duration = end_t - start_t
            
            ms = final_sol.calculate_makespan()
            feasible, _ = final_sol.is_feasible()
            
            if not feasible:
                ms = float('inf')
            
            makespans.append(ms)
            runtimes.append(duration)
            
            if ms < best_makespan_instance:
                best_makespan_instance = ms
                best_sol_object = final_sol.copy()
        
        avg_makespan = statistics.mean(makespans)
        avg_runtime = statistics.mean(runtimes)
        
        # Save best solution
        if best_sol_object:
            sol_filename = instance_name.replace(".txt", "_sol.txt")
            sol_path = os.path.join(solution_output_dir, sol_filename)
            best_sol_object.save_to_file(sol_path)
            
        print(f"✓ {instance_name:<30} | Best: {best_makespan_instance:<10.2f} | Avg: {avg_makespan:<10.2f} | Time: {avg_runtime:.2f}s")
        
        return {
            "Instance": instance_name,
            "Best Objective": best_makespan_instance,
            "Avg Objective": avg_makespan,
            "Avg Time (s)": avg_runtime
        }
    except Exception as e:
        print(f"✗ ERROR {filepath}: {e}")
        return None


def run_benchmark(data_folder: str, output_csv: str, solution_dir: str, 
                  num_rounds: int = 10, num_iters: int = 200, max_workers: int = None):
    """
    Chạy benchmark song song trên tất cả instances.
    """
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    os.makedirs(solution_dir, exist_ok=True)
    
    instance_files = sorted(glob.glob(os.path.join(data_folder, "*.txt")))
    
    if not instance_files:
        print(f"Không tìm thấy file .txt trong {data_folder}")
        return None
    
    print(f"🚀 Bắt đầu benchmark {len(instance_files)} instances...")
    print(f"   Rounds: {num_rounds} | Iterations: {num_iters} | Workers: {max_workers or 'Auto'}")
    print("-" * 80)
    
    results = []
    
    # Parallel execution
    with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(process_instance, f, solution_dir, num_rounds, num_iters): f 
            for f in instance_files
        }
        
        for future in concurrent.futures.as_completed(futures):
            res = future.result()
            if res:
                results.append(res)
    
    # Sort and create DataFrame
    results.sort(key=lambda x: x['Instance'])
    df = pd.DataFrame(results)
    
    if not df.empty:
        avg_row = {
            "Instance": "AVERAGE",
            "Best Objective": df["Best Objective"].mean(),
            "Avg Objective": df["Avg Objective"].mean(),
            "Avg Time (s)": df["Avg Time (s)"].mean()
        }
        df = pd.concat([df, pd.DataFrame([avg_row])], ignore_index=True)
    
    df.to_csv(output_csv, index=False)
    
    print("-" * 80)
    print(f"✅ Hoàn tất! Kết quả lưu tại: {output_csv}")
    print(f"   Solutions lưu tại: {solution_dir}")
    
    return df

print("Hàm benchmark đã sẵn sàng!")

## 7. 🎯 CHẠY BENCHMARK

**Tùy chỉnh các tham số bên dưới:**

In [ ]:
# ============== CẤU HÌNH ==============
DATA_FOLDER = "./data/0130/10_instances"    # Thư mục chứa dữ liệu test
OUTPUT_CSV = "./output/benchmark_results.csv"  # File kết quả
SOLUTION_DIR = "./output/solutions"         # Thư mục lưu solutions

NUM_ROUNDS = 10     # Số lần chạy mỗi instance
NUM_ITERS = 200     # Số vòng lặp Tabu Search
MAX_WORKERS = 2     # Số workers song song (Colab có 2 CPU cores)
# ======================================

# Chạy benchmark
results_df = run_benchmark(
    data_folder=DATA_FOLDER,
    output_csv=OUTPUT_CSV,
    solution_dir=SOLUTION_DIR,
    num_rounds=NUM_ROUNDS,
    num_iters=NUM_ITERS,
    max_workers=MAX_WORKERS
)

## 8. Xem kết quả

In [ ]:
if results_df is not None:
    display(results_df)
else:
    print("Chưa có kết quả. Hãy chạy benchmark trước.")

## 9. Tải kết quả về máy

In [ ]:
from google.colab import files

# Tải file CSV
files.download(OUTPUT_CSV)

# Nén và tải thư mục solutions
!zip -r solutions.zip {SOLUTION_DIR}
files.download('solutions.zip')

## 10. (Tùy chọn) Lưu kết quả lên Drive

In [ ]:
# Copy kết quả lên Drive
DRIVE_OUTPUT = "/content/drive/MyDrive/benchmark_results"
!mkdir -p "{DRIVE_OUTPUT}"
!cp {OUTPUT_CSV} "{DRIVE_OUTPUT}/"
!cp -r {SOLUTION_DIR} "{DRIVE_OUTPUT}/"
print(f"Đã lưu kết quả lên Drive: {DRIVE_OUTPUT}")